<small>
# ETL Process for DICOM Metadata: Extract, Transform, Load

## Overview
This notebook outlines the **ETL (Extract, Transform, Load)** process for handling DICOM metadata. The process begins with **extracting** metadata from the staged DICOM files, followed by **transforming** the data to enrich it with additional information such as slice metadata, and finally **loading** the data into an SQLite database.

Once the data is loaded into the database, the notebook displays the tables, providing a clear overview of the results.

## Process
1. **Extract Metadata**: Metadata is extracted from the staged DICOM files, including patient information, study data, and series details.
2. **Transform Data**: Additional data transformations are applied, such as extracting slice information and adding it to the metadata.
3. **Load Data into SQLite**: The extracted and transformed data is loaded into an SQLite database for further analysis or storage.
4. **Display Data**: The populated tables in the SQLite database are displayed in a readable format.

## Inputs
- **Staged DICOM Files**: DICOM files already copied into the staging location.
  
## Outputs
- **SQLite Database**: Populated tables with metadata, including patient, study, series, and slice information.
- **Pretty Table Display**: Visual representation of the tables in the database for validation.

In [ ]:
import os
import logging
import sqlite3
import pydicom
from pathlib import Path

class Prep_Dicom_Db:
    def __init__(self, db_file):
        self.db_file = db_file
        try:
            self.conn = sqlite3.connect(db_file)
            self.cursor = self.conn.cursor()

            # WAL mode for performance <write ahead logging>
            self.cursor.execute('PRAGMA journal_mode=WAL;')

            self.create_tables()
        except sqlite3.Error as e:
            logging.error(f"Failed to connect to database {db_file}: {e}")
            raise  

    def create_tables(self):
        """Create necessary tables in the database if they don't already exist."""
        try:
            # Create Patient Table
            # self.cursor.execute('''DROP TABLE IF EXISTS Patient''')
            self.cursor.execute('''CREATE TABLE IF NOT EXISTS Patient (
                                    PatientID TEXT PRIMARY KEY,
                                    PatientName TEXT,
                                    PatientSex TEXT,
                                    PatientAge TEXT,
                                    PatientBirthDate TEXT,
                                    PatientIdentityRemoved TEXT,
                                    DeIdentificationMethod TEXT,
                                    LastMenstrualDate TEXT
                                )''')

            # Create Study Table
            # self.cursor.execute('''DROP TABLE IF EXISTS Study''')
            self.cursor.execute('''CREATE TABLE IF NOT EXISTS Study (
                                    StudyInstanceUID TEXT PRIMARY KEY,
                                    PatientID TEXT,
                                    StudyDescription TEXT,
                                    StudyDate TEXT,
                                    StudyTime TEXT,
                                    StudyID TEXT,
                                    SeriesDate TEXT,
                                    ReferringPhysicianName TEXT,
                                    FOREIGN KEY (PatientID) REFERENCES Patient (PatientID)
                                )''')

            # Create Series Table
            #self.cursor.execute('''DROP TABLE IF EXISTS Series''')
            self.cursor.execute('''CREATE TABLE IF NOT EXISTS Series (
                                    SeriesInstanceUID TEXT PRIMARY KEY,
                                    StudyInstanceUID TEXT,
                                    SeriesDescription TEXT,
                                    Modality TEXT,
                                    SliceThickness TEXT,
                                    PixelSpacing TEXT,
                                    ImagePositionPatient TEXT,
                                    ImageOrientationPatient TEXT,
                                    Rows INTEGER,
                                    Columns INTEGER,
                                    ImageType TEXT,
                                    AcquisitionNumber TEXT,
                                    InstanceNumber TEXT,
                                    PatientPosition TEXT,
                                    ExposureTime INTEGER,
                                    KVP REAL,
                                    TableHeight REAL,
                                    SliceLocation REAL,
                                    BodyPartExamined TEXT,
                                    ScanOptions TEXT,
                                    ReconstructionDiameter REAL,
                                    DistanceSourceToPatient REAL,
                                    GantryDetectorTilt REAL,
                                    RotationDirection TEXT,
                                    ConvolutionKernel TEXT,
                                    FOREIGN KEY (StudyInstanceUID) REFERENCES Study (StudyInstanceUID)
                                )''')

            # Create SliceMetadata Table
            # self.cursor.execute('''DROP TABLE IF EXISTS SliceMetadata''')
            self.cursor.execute('''CREATE TABLE IF NOT EXISTS SliceMetadata (
                                    SliceID INTEGER PRIMARY KEY AUTOINCREMENT,
                                    SeriesInstanceUID TEXT,
                                    AcquisitionDate TEXT,
                                    AcquisitionTime TEXT,
                                    ContrastBolusAgent TEXT,
                                    AccessionNumber TEXT,
                                    SliceNumber INTEGER,
                                    SliceLocation REAL,
                                    SliceThickness REAL,
                                    PixelPaddingValue INTEGER,
                                    RescaleIntercept REAL,
                                    RescaleSlope REAL,
                                    LongitudinalTemporalInfo TEXT,
                                    FOREIGN KEY (SeriesInstanceUID) REFERENCES Series (SeriesInstanceUID)
                                )''')

            self.conn.commit()  
            print("Tables created successfully")
        except sqlite3.Error as e:
            logging.error(f"Error creating tables: {e}")
            raise

    def convert_to_db_value(self, value):
        """Convert values to appropriate types that SQLite can handle."""
        if value is None:
            return None
        if isinstance(value, list):
            return ','.join(map(str, value)) 
        if isinstance(value, (int, float)):
            return value
        return str(value) 

    def insert_patient(self, patient_metadata_list):
        """Insert patient data into the database."""
        print("----Inserting Patients data-----")
        if patient_metadata_list:
            patient_metadata_list_converted = [
                (
                    self.convert_to_db_value(metadata[0]),  # PatientID
                    self.convert_to_db_value(metadata[1]),  # PatientName
                    self.convert_to_db_value(metadata[2]),  # PatientSex
                    self.convert_to_db_value(metadata[3]),  # PatientAge
                    self.convert_to_db_value(metadata[4]),  # PatientBirthDate
                    self.convert_to_db_value(metadata[5]),  # PatientIdentityRemoved
                    self.convert_to_db_value(metadata[6]),  # DeIdentificationMethod
                    self.convert_to_db_value(metadata[7])   # LastMenstrualDate
                ) for metadata in patient_metadata_list
            ]

            self.cursor.executemany('''INSERT OR IGNORE INTO Patient (
                PatientID, PatientName, PatientSex, PatientAge, PatientBirthDate, 
                PatientIdentityRemoved, DeIdentificationMethod, LastMenstrualDate
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)''', patient_metadata_list_converted)
            self.conn.commit()

    def insert_study_batch(self, study_metadata_list):
        print("----Inserting Study data-----")

        if study_metadata_list:
            # Apply convert_to_db_value to all the values before insertion
            study_metadata_list_converted = [
            (
                self.convert_to_db_value(metadata[0]),  # StudyInstanceUID
                self.convert_to_db_value(metadata[1]),  # PatientID
                self.convert_to_db_value(metadata[2]),  # StudyDescription
                self.convert_to_db_value(metadata[3]),  # StudyDate
                self.convert_to_db_value(metadata[4]),  # StudyTime
                self.convert_to_db_value(metadata[5]),  # StudyID
                self.convert_to_db_value(metadata[6]),  # SeriesDate
                self.convert_to_db_value(metadata[7])   # ReferringPhysicianName
            ) for metadata in study_metadata_list
            ]
            self.cursor.executemany('''INSERT OR IGNORE INTO Study (
                                        StudyInstanceUID, PatientID, StudyDescription, StudyDate, StudyTime,
                                        StudyID, SeriesDate, ReferringPhysicianName
                                    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)''', 
                                    study_metadata_list_converted)
            self.conn.commit()

    def insert_series_batch(self, series_metadata_list):
        print("----Inserting Series data-----")
        if series_metadata_list:
            #  number of slices per series
            all_series =[]
            if series_metadata_list:

                metadata_with_slices_converted = [(
                self.convert_to_db_value(metadata[0]),  # SeriesInstanceUID
                self.convert_to_db_value(metadata[1]),  # StudyInstanceUID
                self.convert_to_db_value(metadata[2]),  # SeriesDescription
                self.convert_to_db_value(metadata[3]),  # Modality
                self.convert_to_db_value(metadata[4]),  # SliceThickness
                self.convert_to_db_value(metadata[5]),  # PixelSpacing
                self.convert_to_db_value(metadata[6]),  # ImagePositionPatient
                self.convert_to_db_value(metadata[7]),  # ImageOrientationPatient
                self.convert_to_db_value(metadata[8]),  # Rows
                self.convert_to_db_value(metadata[9]),  # Columns
                self.convert_to_db_value(metadata[10]), # ImageType
                self.convert_to_db_value(metadata[11]), # AcquisitionNumber
                self.convert_to_db_value(metadata[12]), # InstanceNumber
                self.convert_to_db_value(metadata[13]), # PatientPosition
                self.convert_to_db_value(metadata[14]), # ExposureTime
                self.convert_to_db_value(metadata[15]), # KVP
                self.convert_to_db_value(metadata[16]), # TableHeight
                self.convert_to_db_value(metadata[17]), # SliceLocation
                self.convert_to_db_value(metadata[18]), # BodyPartExamined
                self.convert_to_db_value(metadata[19]), # ScanOptions
                self.convert_to_db_value(metadata[20]), # ReconstructionDiameter
                self.convert_to_db_value(metadata[21]), # DistanceSourceToPatient
                self.convert_to_db_value(metadata[22]), # GantryDetectorTilt
                self.convert_to_db_value(metadata[23]), # RotationDirection
                self.convert_to_db_value(metadata[24]), # ConvolutionKernel
                ) for metadata in series_metadata_list]

            self.cursor.executemany('''INSERT OR IGNORE INTO Series (
                                        SeriesInstanceUID, StudyInstanceUID, SeriesDescription, Modality, SliceThickness,
                                        PixelSpacing, ImagePositionPatient, ImageOrientationPatient, Rows, Columns,
                                        ImageType, AcquisitionNumber, InstanceNumber, PatientPosition, ExposureTime,
                                        KVP, TableHeight, SliceLocation, BodyPartExamined, ScanOptions, 
                                        ReconstructionDiameter, DistanceSourceToPatient, GantryDetectorTilt, 
                                        RotationDirection, ConvolutionKernel
                                    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ? ,?)''',
                                    metadata_with_slices_converted)
            self.conn.commit()

    def insert_slice_metadata_batch(self, slice_metadata_list):
        print("----Inserting Slice meta data-----")
        if slice_metadata_list:
            slice_metadata_list_converted = [
                (
                    self.convert_to_db_value(metadata[0]),  # SeriesInstanceUID
                    self.convert_to_db_value(metadata[1]),  # AcquisitionDate
                    self.convert_to_db_value(metadata[2]),  # AcquisitionTime
                    self.convert_to_db_value(metadata[3]),  # ContrastBolusAgent
                    self.convert_to_db_value(metadata[4]),  # AccessionNumber
                    self.convert_to_db_value(metadata[5]),  # SliceNumber
                    self.convert_to_db_value(metadata[6]),  # SliceLocation
                    self.convert_to_db_value(metadata[7]),  # SliceThickness
                    self.convert_to_db_value(metadata[8]),  # PixelPaddingValue
                    self.convert_to_db_value(metadata[9]),  # RescaleIntercept
                    self.convert_to_db_value(metadata[10]), # RescaleSlope
                    self.convert_to_db_value(metadata[11])  # LongitudinalTemporalInfo
                ) for metadata in slice_metadata_list
            ]
            self.cursor.executemany('''INSERT INTO SliceMetadata (
                                        SeriesInstanceUID, AcquisitionDate, AcquisitionTime, 
                                        ContrastBolusAgent, AccessionNumber, SliceNumber, SliceLocation, 
                                        SliceThickness, PixelPaddingValue, RescaleIntercept, 
                                        RescaleSlope, LongitudinalTemporalInfo
                                    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''',
                                    slice_metadata_list_converted)
            self.conn.commit()

    def parse_dicom_header(self, dicom_file):
        """Parse relevant fields from a DICOM file and return the metadata."""  
        try:
            dicom_data = pydicom.dcmread(dicom_file)
            metadata = {
                'PatientID': dicom_data.PatientID,
                'PatientName': dicom_data.get('PatientName', 'N/A'),
                'PatientSex': dicom_data.get('PatientSex', 'N/A'),
                'PatientAge': dicom_data.get('PatientAge', 'N/A'),
                'PatientBirthDate': dicom_data.get('PatientBirthDate', 'N/A'),
                'PatientIdentityRemoved': dicom_data.get('PatientIdentityRemoved', 'N/A'),
                'DeIdentificationMethod': dicom_data.get('DeIdentificationMethod', 'N/A'),
                'LastMenstrualDate': dicom_data.get('LastMenstrualDate', 'N/A'),
                'StudyInstanceUID': dicom_data.StudyInstanceUID,
                'StudyDescription': dicom_data.get('StudyDescription', 'N/A'),
                'StudyDate': dicom_data.get('StudyDate', 'N/A'),
                'StudyTime': dicom_data.get('StudyTime', 'N/A'),
                'StudyID': dicom_data.get('StudyID', 'N/A'),
                'SeriesDate': dicom_data.get('SeriesDate', 'N/A'),
                'AccessionNumber': dicom_data.get('AccessionNumber', 'N/A'),
                'ReferringPhysicianName': dicom_data.get('ReferringPhysicianName', 'N/A'),
                'SeriesInstanceUID': dicom_data.SeriesInstanceUID,
                'SeriesDescription': dicom_data.get('SeriesDescription', 'N/A'),
                'Modality': dicom_data.Modality,
                'SliceThickness': dicom_data.get('SliceThickness', 'N/A'),
                'PixelSpacing': dicom_data.get('PixelSpacing', 'N/A'),
                'ImagePositionPatient': dicom_data.get('ImagePositionPatient', 'N/A'),
                'ImageOrientationPatient': dicom_data.get('ImageOrientationPatient', 'N/A'),
                'Rows': dicom_data.get('Rows', 'N/A'),
                'Columns': dicom_data.get('Columns', 'N/A'),
                'ImageType': dicom_data.get('ImageType', 'N/A'),
                'AcquisitionNumber': dicom_data.get('AcquisitionNumber', 'N/A'),
                'InstanceNumber': dicom_data.get('InstanceNumber', 'N/A'),
                'PatientPosition': dicom_data.get('PatientPosition', 'N/A'),
                'ExposureTime': dicom_data.get('ExposureTime', 'N/A'),
                'KVP': dicom_data.get('KVP', 'N/A'),
                'TableHeight': dicom_data.get('TableHeight', 'N/A'),
                'SliceLocation': dicom_data.get('SliceLocation', 'N/A'),
                'BodyPartExamined': dicom_data.get('BodyPartExamined', 'N/A'),
                'ScanOptions': dicom_data.get('ScanOptions', 'N/A'),
                'ReconstructionDiameter': dicom_data.get('ReconstructionDiameter', 'N/A'),
                'DistanceSourceToPatient': dicom_data.get('DistanceSourceToPatient', 'N/A'),
                'GantryDetectorTilt': dicom_data.get('GantryDetectorTilt', 'N/A'),
                'RotationDirection': dicom_data.get('RotationDirection', 'N/A'),
                'ConvolutionKernel': dicom_data.get('ConvolutionKernel', 'N/A'),
                'AcquisitionDate': dicom_data.get('AcquisitionDate', 'N/A'),  
                'AcquisitionTime': dicom_data.get('AcquisitionTime', 'N/A'),  
                'ContrastBolusAgent': dicom_data.get('ContrastBolusAgent', 'N/A'),
                'SliceNumber': dicom_data.get('SliceNumber', 'N/A'),
                'SliceLocation': dicom_data.get('SliceLocation', 'N/A'),
                'PixelPaddingValue': dicom_data.get('PixelPaddingValue', 'N/A'),
                'RescaleIntercept': dicom_data.get('RescaleIntercept', 'N/A'),
                'RescaleSlope': dicom_data.get('RescaleSlope', 'N/A'),
                'LongitudinalTemporalInfo': dicom_data.get('LongitudinalTemporalInfo', 'N/A')
            }

            return metadata

        except Exception as e:
            logging.error(f"Error parsing DICOM file {dicom_file}: {e}")
            return None
    def process_dicom_folder(self, folder_path):
            """Process all DICOM files in a folder and its subfolders."""
            if not os.path.isdir(folder_path):
                logging.error(f"{folder_path} is not a valid directory.")
                return
            
            dicom_files = []
            # Walking through all subdirectories and files
            for dirpath, _, filenames in os.walk(folder_path):
                dicom_files.extend([os.path.join(dirpath, f) for f in filenames if f.endswith('.dcm')])
            
            if not dicom_files:
                logging.error(f"No DICOM files found in {folder_path}.")

            patient_metadata_list= []
            study_metadata_list = []
            series_metadata_list = []
            slice_metadata_list = []

            for dicom_file in dicom_files:
                print(dicom_file)
                dicom_metadata = self.parse_dicom_header(dicom_file)
                if dicom_metadata:
                    patient_metadata_list.append((
                        dicom_metadata['PatientID'],
                        dicom_metadata['PatientName'],dicom_metadata['PatientSex'],
                        dicom_metadata['PatientAge'], dicom_metadata['PatientBirthDate'],
                        dicom_metadata['PatientIdentityRemoved'],dicom_metadata['DeIdentificationMethod'],
                        dicom_metadata['LastMenstrualDate']
                    ))
                    study_metadata_list.append((
                        dicom_metadata['StudyInstanceUID'], dicom_metadata['PatientID'],
                        dicom_metadata['StudyDescription'], dicom_metadata['StudyDate'], dicom_metadata['StudyTime'],
                        dicom_metadata['StudyID'], dicom_metadata['SeriesDate'], dicom_metadata['ReferringPhysicianName']
                    ))

                    series_metadata_list.append((
                        dicom_metadata['SeriesInstanceUID'], dicom_metadata['StudyInstanceUID'],
                        dicom_metadata['SeriesDescription'], dicom_metadata['Modality'], dicom_metadata['SliceThickness'],
                        dicom_metadata['PixelSpacing'], dicom_metadata['ImagePositionPatient'], dicom_metadata['ImageOrientationPatient'],
                        dicom_metadata['Rows'], dicom_metadata['Columns'], dicom_metadata['ImageType'],
                        dicom_metadata['AcquisitionNumber'], dicom_metadata['InstanceNumber'], dicom_metadata['PatientPosition'],
                        dicom_metadata['ExposureTime'], dicom_metadata['KVP'], dicom_metadata['TableHeight'],
                        dicom_metadata['SliceLocation'], dicom_metadata['BodyPartExamined'], dicom_metadata['ScanOptions'],
                        dicom_metadata['ReconstructionDiameter'], dicom_metadata['DistanceSourceToPatient'], dicom_metadata['GantryDetectorTilt'],
                        dicom_metadata['RotationDirection'], dicom_metadata['ConvolutionKernel']
                    ))

                    slice_metadata_list.append((
                        dicom_metadata['SeriesInstanceUID'], dicom_metadata['AcquisitionDate'],
                        dicom_metadata['AcquisitionTime'], dicom_metadata['ContrastBolusAgent'],
                        dicom_metadata['AccessionNumber'], dicom_metadata['SliceNumber'], dicom_metadata['SliceLocation'],
                        dicom_metadata['SliceThickness'], dicom_metadata['PixelPaddingValue'], dicom_metadata['RescaleIntercept'],
                        dicom_metadata['RescaleSlope'], dicom_metadata['LongitudinalTemporalInfo']
                    ))

            # bulk insertion
            self.insert_patient(patient_metadata_list)
            self.insert_study_batch(study_metadata_list)
            self.insert_slice_metadata_batch(slice_metadata_list)
            self.insert_series_batch(series_metadata_list)
            

def main():
    # Get base directory and resolve paths
    base_dir = Path(os.getcwd()).resolve()
    dicom_folder = base_dir / 'qure_ai' / 'lidc_small_dset_staging'
    db_file = base_dir / 'dicom_data.db'

    # Convert paths to string and normalize for cross-platform compatibility
    base_dir = str(base_dir).replace("\\", "/")
    dicom_folder = str(dicom_folder).replace("\\", "/")
    db_file = str(db_file).replace("\\", "/")
    
    # Initialize the database and process the DICOM folder
    dicom_db = Prep_Dicom_Db(db_file)
    dicom_db.process_dicom_folder(dicom_folder)

if __name__ == '__main__':
    main()


#####  display data stored in the tables

Adding Number of Slices per Series

In [ ]:
import sqlite3

def update_number_of_slices(db_path):
    connection = sqlite3.connect(db_path)
    cursor = connection.cursor()
    
    try:
        
        cursor.execute('''
            ALTER TABLE Series ADD COLUMN NumberOfSlices INTEGER
        ''')
        
        # Update the NumberOfSlices column with the count of slices per SeriesInstanceUID
        cursor.execute('''
            UPDATE Series
            SET NumberOfSlices = (
                SELECT COUNT(*)
                FROM SliceMetadata
                WHERE SliceMetadata.SeriesInstanceUID = Series.SeriesInstanceUID
            )
        ''')
        
        connection.commit()
        
        print("NumberOfSlices updated successfully.")

    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
        connection.rollback()

    finally:
        cursor.close()
        connection.close()

db_path = 'dicom_data.db' 
update_number_of_slices(db_path)


NumberOfSlices updated successfully.


#####  display data stored in the tables

In [54]:
import sqlite3
import pandas as pd
from tabulate import tabulate

class DicomDbBase:
    def __init__(self, db_file):
        self.conn = sqlite3.connect(db_file)
        self.cursor = self.conn.cursor()

    def fetch_table_data(self, table_name,limit):
        self.cursor.execute(f"SELECT * FROM {table_name} LIMIT {limit};")
        rows = self.cursor.fetchall()

        columns = [description[0] for description in self.cursor.description]
        table_df = pd.DataFrame(rows, columns=columns)  #pandas df

        return table_df
    
    def print_pretty_table_data(self, table_name,limit):
        table_df = self.fetch_table_data(table_name,limit)
        print(f"\n{table_name} Table:")
        print(tabulate(table_df, headers='keys', tablefmt='pretty', showindex=False))

    def close(self):
        self.conn.close()


class Show_Dicom_db(DicomDbBase):
    """ Class for showing DICOM data from Study and Series tables """
    def __init__(self, db_file):
        super().__init__(db_file)  


class Display_Dicom_Db(DicomDbBase):
    """ Class for displaying DICOM data from Patient and SliceMetadata tables """
    def __init__(self, db_file):
        super().__init__(db_file)  


db_file = 'dicom_data.db'  
dicom_db_show = Show_Dicom_db(db_file)

# tables in the db
tables_show = ['Study', 'Series']
for table in tables_show:
    dicom_db_show.print_pretty_table_data(table, limit=100)

dicom_db_show.close()



Study Table:
+------------------------------------------------------------------+----------------+------------------+-----------+-----------+---------+------------+------------------------+
|                         StudyInstanceUID                         |   PatientID    | StudyDescription | StudyDate | StudyTime | StudyID | SeriesDate | ReferringPhysicianName |
+------------------------------------------------------------------+----------------+------------------+-----------+-----------+---------+------------+------------------------+
| 1.3.6.1.4.1.14519.5.2.1.6279.6001.193241055656414949090207821605 | LIDC-IDRI-0400 |       N/A        | 20000101  |           |         |  20000101  |                        |
| 1.3.6.1.4.1.14519.5.2.1.6279.6001.344370459068774891776634727699 | LIDC-IDRI-0401 |  CT LUNG SCREEN  | 20000101  |  172247   |         |  20000101  |                        |
| 1.3.6.1.4.1.14519.5.2.1.6279.6001.156618458422978822822335971869 | LIDC-IDRI-0402 | CT CHEST O CONT

In [55]:
# Create instances for Display_Dicom_Db
dicom_db_display = Display_Dicom_Db(db_file)

# tables in the db
tables_display = ['Patient', 'SliceMetadata']
for table in tables_display:
    dicom_db_display.print_pretty_table_data(table, limit=100)

dicom_db_display.close()


Patient Table:
+----------------+-------------+------------+------------+------------------+------------------------+------------------------+-------------------+
|   PatientID    | PatientName | PatientSex | PatientAge | PatientBirthDate | PatientIdentityRemoved | DeIdentificationMethod | LastMenstrualDate |
+----------------+-------------+------------+------------+------------------+------------------------+------------------------+-------------------+
| LIDC-IDRI-0400 |             |            |    N/A     |                  |          YES           |          N/A           |     20000101      |
| LIDC-IDRI-0401 |             |            |    N/A     |                  |          YES           |          N/A           |     20000101      |
| LIDC-IDRI-0402 |             |            |    N/A     |                  |          YES           |          N/A           |     20000101      |
| LIDC-IDRI-0403 |             |            |            |                  |          YES      